In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model.gpt import GPT
from pathlib import Path
import torch
from fastai.learner import *
from fastai.text.all import *

/home/agrawalp2/miniconda3/envs/tinylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
model_identifier = 'meta-llama/Meta-Llama-3-8B'
# model_identifier = 'microsoft/Phi-3-mini-4k-instruct'

In [4]:
# model = AutoModelForCausalLM.from_pretrained(model_identifier, trust_remote_code = True)
model = GPT.from_hf(model_identifier, enable_qlora= True)

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.54s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model.block_size

4096

In [5]:
inps = torch.ones(1,200).cuda().long()

SyntaxError: invalid syntax (2422320692.py, line 1)

In [ ]:
out = model(inps)

In [ ]:
out

In [ ]:
from torch.amp import autocast

with autocast():
    out = model(inps)

In [ ]:
??model.__call__